In [8]:
import csv
from datetime import datetime
import pandas as pd

#Define file paths
big_data_file = '/home/ubuntu/myraidstorage/cryptobot_cs554_project/big-btc-sentiment-time-series.csv'
btcusd_file = '/home/ubuntu/myraidstorage/cryptobot_cs554_project/btcusd-2018-2023-copy.csv'
jan18_sentiment = '/home/ubuntu/myraidstorage/cryptobot_cs554_project/RS_2018-01-cleaned_updated_sentiment_analysis_new.csv'
feb18_sentiment = '/home/ubuntu/myraidstorage/cryptobot_cs554_project/RS_2018-02-cleaned_updated_sentiment_analysis_new.csv'
march18_sentiment = '/home/ubuntu/myraidstorage/cryptobot_cs554_project/RS_2018-03-cleaned_updated_sentiment_analysis_new.csv'

In [6]:
#----------------- Step 1: Create model_data.csv and write header row-----------------#
with open(big_data_file, mode='w', newline='') as model_file:
    writer = csv.writer(model_file)
    writer.writerow(['date', 'reddit_sentiment_aggregate', 'positive_post_volume', 'negative_posts_volume',
                     'positive_sentiment_volume_change', 'negative_sentiment_volume_change', 'BTC_trade_volume',
                     'BTC_closing_price', 'BTC_price_change', 'SP500_price', 'SP500_price_change', 'crude_oil_price',
                     'crude_oil_price_change'])


In [12]:
#-----------------Step 2: Add BTC data from btcusd-2018-2023.csv for days 1-Jan-18 through 31-March-18-----------------#
with open(btcusd_file) as btc_file, \
        open(big_data_file, mode='r', newline='') as model_file:
    reader = csv.reader(btc_file)
    next(reader)  # skip header row
    next(model_file)  # skip header row
    model_reader = csv.reader(model_file)
    model_rows = list(model_reader)
    btc_rows = list(reader)
    j = 0
    for i in range(len(model_rows)):
        model_date = datetime.strptime(model_rows[i][0], '%d-%b-%y')
        btc_date = datetime.strptime(btc_rows[j][0], '%Y-%m-%d %H:%M:%S')
        while btc_date < model_date:
            j += 1
            btc_date = datetime.strptime(btc_rows[j][0], '%Y-%m-%d %H:%M:%S')
        model_rows[i][7] = btc_rows[j][4]  # BTC closing price
        if i > 0:
            model_rows[i][8] = str(float(model_rows[i][7]) - float(model_rows[i-1][7]))  # BTC price change

   
    # write updated rows to model_data.csv
    with open(big_data_file, mode='a', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerows(model_rows)

IndexError: list index out of range

In [ ]:

#----------------- Step 3: Fill in SP500 data -----------------#
with open('sp500.csv') as sp500_file, \
        open('model_data.csv', mode='r', newline='') as model_file:
    reader = csv.reader(sp500_file)
    next(reader)  # skip header row
    next(model_file)  # skip header row
    model_reader = csv.reader(model_file)
    model_rows = list(model_reader)
    sp500_rows = list(reader)
    for i in range(len(model_rows)):
        model_date = datetime.strptime(model_rows[i][0], '%m/%d/%Y').strftime('%Y-%m-%d')
        for j in range(len(sp500_rows)):
            sp500_date = datetime.strptime(sp500_rows[j][0], '%Y-%m-%d').strftime('%m/%d/%Y')
            if sp500_date == model_date:
                model_rows[i][9] = sp500_rows[j][1]
                break
    with open('model_data.csv', mode='w', newline='') as model_file:
        writer = csv.writer(model_file)
        writer.writerows(model_rows)
        
    # Calculate SP500 price change
    for i in range(len(model_rows)):
        if i == 0:
            model_rows[i].append(None)
        else:
            prev_price = float(model_rows[i-1][9])
            curr_price = float(model_rows[i][9])
            model_rows[i].append(curr_price - prev_price)
    with open('model_data.csv', mode='w', newline='') as model_file:
        writer = csv.writer(model_file)
        writer.writerows(model_rows)


In [5]:

#-----------------Step 4-----------------#
import csv
from datetime import datetime

with open('crude_oil_price.csv') as crude_oil_file, \
        open('model_data.csv', mode='r', newline='') as model_file:
    reader = csv.reader(crude_oil_file)
    next(reader)  # skip header row
    next(model_file)  # skip header row
    model_reader = csv.reader(model_file)
    model_rows = list(model_reader)
    crude_oil_rows = list(reader)
    for i in range(len(model_rows)):
        model_date = datetime.strptime(model_rows[i][0], '%Y-%m-%d').date()
        crude_oil_date = datetime.strptime(crude_oil_rows[i][0], '%m/%d/%Y').date()
        while crude_oil_date < model_date:
            # move to the next row in the crude oil data until we reach a date that's equal to or later than the model date
            crude_oil_rows.pop(0)
            crude_oil_date = datetime.strptime(crude_oil_rows[0][0], '%m/%d/%Y').date()
        if crude_oil_date == model_date:
            # if we found the matching date, fill in the crude oil price and crude oil price change columns
            model_rows[i][10] = crude_oil_rows[0][1]
            if i > 0:
                prev_crude_oil_price = float(crude_oil_rows[0][1])
                curr_crude_oil_price = float(crude_oil_rows[1][1])
                crude_oil_price_change = (curr_crude_oil_price - prev_crude_oil_price) / prev_crude_oil_price
                model_rows[i][11] = crude_oil_price_change
            crude_oil_rows.pop(0)
        else:
            # if we didn't find the matching date, print a warning message and leave the crude oil price and crude oil price change columns blank
            print(f'Warning: No crude oil price data for {model_date}')
    with open('model_data.csv', mode='w', newline='') as outfile:
        writer = csv.writer(outfile)
        writer.writerow(['date', 'reddit_sentiment_aggregate', 'positive_post_volume', 'negative_posts_volume',
                         'positive_sentiment_volume_change', 'negative_sentiment_volume_change', 'BTC_trade_volume',
                         'BTC_closing_price', 'BTC_price_change', 'SP500_price', 'SP500_price_change', 'crude_oil_price',
                         'crude_oil_price_change'])
        for row in model_rows:
            writer.writerow(row)

In [19]:
import numpy as np
#-----------------Step 5-----------------#
# Load sentiment data
sentiment_files = [jan18_sentiment, feb18_sentiment, march18_sentiment]
sentiment_dfs = []
for sentiment_file in sentiment_files:
    df = pd.read_csv(sentiment_file)
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s').dt.date
    sentiment_dfs.append(df)
sentiment_df = pd.concat(sentiment_dfs, ignore_index=True)

# # Calculate daily sentiment aggregate
# sentiment_df['sentiment_value'] = sentiment_df['predicted_category'].map({'bearish': -1, 'bullish': 1, 'neutral': 0})
# sentiment_df['weighted_sentiment_value'] = sentiment_df['sentiment_value'] * sentiment_df['confidence']

# Aggregate sentiment to daily
daily_sentiment['reddit_sentiment_aggregate'] = sentiment_df.groupby('timestamp').apply(lambda x: np.average(x['predicted_category'].map({'bearish': -1, 'bullish': 1, 'neutral': 0}), weights=x['confidence']))
daily_sentiment['timestamp'] = pd.to_datetime(daily_sentiment['timestamp'])
daily_sentiment.set_index('timestamp', inplace=True)

# Calculate post volume
positive_post_volume = sentiment_df[sentiment_df['predicted_category'] == 'bullish'].groupby('timestamp').size().rename('positive_post_volume')
negative_post_volume = sentiment_df[sentiment_df['predicted_category'] == 'bearish'].groupby('timestamp').size().rename('negative_post_volume')
daily_sentiment = daily_sentiment.join([positive_post_volume, negative_post_volume], how='outer')

# Fill in missing days
start_date = sentiment_df['timestamp'].min()
end_date = sentiment_df['timestamp'].max()
date_range = pd.date_range(start=start_date, end=end_date, freq='D')
daily_sentiment = daily_sentiment.reindex(date_range, fill_value=0)

# Save data to CSV
daily_sentiment.to_csv(big_data_file, index_label='date')



KeyError: 'timestamp'